In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp pod.client

# Pod Client

In [ ]:
# export
from integrators.itembase import Edge
from integrators.schema import *
from integrators.imports import *

In [ ]:
# export
API_URL = "http://localhost:3030/v2"

In [ ]:
# export
class podClient:

    def __init__(self, url=API_URL, database_key=None, owner_key=None):
        self.url = url
        self.base_url = f"{url}/{owner_key}"
        self.test_connection()
        self.database_key=database_key
        self.owner_key=owner_key
        # self.local_db = DB()

    def test_connection(self):
        try:
            res = requests.get(self.url)
            return True
        except requests.exceptions.RequestException as e:
            print("Could no connect to backend")
            return False

#     def uid(self):
#         # TODO: REFACTOR
#         return int(1e5 + random.randint(0, 1e5))
    
    def create(self, node):
        if node.uid is None:
            print(f"Error, node {node} has no uid, not creating")
        try:
            body = {  "databaseKey": self.database_key, "payload":self.get_properties_json(node) }

            result = requests.post(f"{self.base_url}/create_item",
                                   json=body)
            if result.status_code != 200:
                print(result, result.content)
                return False
            else:
                return True
        except requests.exceptions.RequestException as e:
            print(e)
            return False
    
    def create_edges(self, edges):
        """Create edges between nodes, edges should be of format [{"_type": "friend", "_source": 1, "_target": 2}]"""
        edges_data = []
        for e in edges:
            src, target = e.source.uid, e.target.uid
            data = {"_source": src, "_target": target, "_type": e._type}
            if e.label is not None: data[LABEL] = e.label
            if e.sequence is not None: data[SEQUENCE] = e.sequence

            if e.reverse:
                data2 = copy(data)
                data2["_source"] = target
                data2["_target"] = src
                data2["_type"] = "~" + data2["_type"]
                edges_data.append(data2)

            edges_data.append(data)

        edges_data = {"databaseKey": self.database_key, "payload": {
                            "createItems": [], "updateItems": [], "createEdges": edges_data}}

        try:
            result = requests.post(f"{self.base_url}/bulk_action",
                                   json=edges_data)
            if result.status_code != 200:
                if "UNIQUE constraint failed" in str(result.content):
                    print(result.status_code, "Edge already exists")
                else:
                    print(result, result.content)
                return False
            else:
                return True
        except requests.exceptions.RequestException as e:
            print(e)
            return False
        
    def create_edge(self, edge):
        return self.create_edges([edge])
    
    def get(self, uid, expanded=True):
        if not expanded:
            return self._get_item_with_properties(uid)
        else:
            return self._get_item_expanded(uid)
    
    def _get_item_expanded(self, uid):
        body = {"payload": [uid],
                "databaseKey": self.database_key}
        try:
            result = requests.post(f"{self.base_url}/get_items_with_edges",
                                    json=body)
            if result.status_code != 200:
                print(result, result.content)
                return None
            else:
                json = result.json()[0]
                res =  self.item_from_json(json)
                return res

        except requests.exceptions.RequestException as e:
            print(e)
            return None

    def _get_item_with_properties(uid):
        try:
            result = requests.get(f"{self.base_url}/items/{uid}")
            if result.status_code != 200:
                print(result, result.content)
                return None
            else:
                json = result.json()
                if json == []:
                    return None
                else:
                    return json
        except requests.exceptions.RequestException as e:
            print(e)
            return None

    def get_properties_json(self, node):
        res = dict()
        for k,v in node.__dict__.items():
            if k[:1] != '_' and not (isinstance(v, list) and len(v)>0 and isinstance(v[0], Edge)) and v is not None:
                res[k] = v
        res["_type"] = node.__class__.__name__
        return res

    def update_item(self, node):
        data = self.get_properties_json(node)
        uid = data["uid"]
        body = {"payload": data,
                "databaseKey": self.database_key}

        try:
            result = requests.post(f"{self.base_url}/update_item",
                                  json=body)
            if result.status_code != 200:
                print(result, result.content)
        except requests.exceptions.RequestException as e:
            print(e)

    def search_by_fields(self, fields_data):

        body = {"payload": fields_data,
                "databaseKey": self.database_key}
        try:
            result = requests.post(f"{self.base_url}/search_by_fields",
                                   json=body)
            json =  result.json()
            return [self.item_from_json(item) for item in json]
        except requests.exceptions.RequestException as e:
            return None

    def item_from_json(self, json):
        indexer_class = json.get("indexerClass", None)
        constructor = get_constructor(json["_type"], indexer_class)
        return constructor.from_json(json)

    def get_properties(self, expanded):
        properties = copy(expanded)
        if ALL_EDGES in properties: del properties[ALL_EDGES]
        return properties

    def run_importer(self, uid, servicePayload):

        body = dict()
        body["databaseKey"] = servicePayload["databaseKey"]
        body["payload"] = {"uid": uid, "servicePayload": servicePayload}

        print(body)

        try:
            res = requests.post(f"{self.base_url}/run_importer", json=body)
            # res = requests.post(self.url)
            if res.status_code != 200:
                print(f"Failed to start importer on {url}:\n{res.status_code}: {res.text}")
            else:
                print("Starting importer")
        except requests.exceptions.RequestException as e:
            print("Error with calling importer {e}")

We communicate with the pod with the PodClient. The PodClient requires us to provide a database key and an owner key. The database Key should be generated by the client once and kept there. The owner key is the full client's public ED25519 key. When you run an integrator, you don't have to worry about these keys, because they are provided by the pod. For testing purposes, we can make our own keys.

In [ ]:
client = podClient(database_key="0" * 64, owner_key="1" * 64)
success = client.test_connection()
assert success

## Creating Items and Edges

Now that we have access to the pod, we can create items here and upload them to the pod.

In [ ]:
email_item = EmailMessage.from_data(content="example content field")
success = client.create(email_item)
assert success
email_item

EmailMessage (#542001)

We can connect items using edges. Let's create another item, a person, and connect the email and the person.

In [ ]:
person_item = Person.from_data(firstName="Alice")
item_succes = client.create(person_item)

edge = Edge(person_item, email_item, "author")
edge_succes = client.create_edge(edge)

assert item_succes and edge_succes

# Fetching and updating Items

We can use the client to fetch data from the database. This is in particular usefull for indexers, which often use data in the database as input for their models. The simplest form  of querying the database is by querying items in the pod by their uid (unique identifier).

In [ ]:
person_item = Person.from_data(firstName="Alice")
client.create(person_item)
person_from_db = client.get(person_item.uid)
assert person_from_db is not None
person_from_db

Person (#542003)

Appart from creating, we might want to update existing items:

In [ ]:
person_item.lastName = "Awesome"
client.update_item(person_item)

person_from_db = client.get(person_item.uid)
assert person_from_db.lastName == "Awesome"
person_from_db

Person (#542003)

Sometimes, we might not know the uids of the items we want to fetch. We can also search by a certain property. A usage examples is when we want to query all items from a particular type

In [ ]:
person_item2 = Person.from_data(firstName="Bob")
client.create(person_item2);
all_people = client.search_by_fields({"_type": "Person"})

assert all([isinstance(p, Person) for p in all_people]) and len(all_people) > 0

all_people[:3]

[Person (#32002), Person (#32004), Person (#32005)]

# Export -

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted basic.ipynb.
Converted index.ipynb.
Converted itembase.ipynb.
Converted pod.client.ipynb.
